# CONFIGURACIONES

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime
from unidecode import unidecode
from fuzzywuzzy import fuzz, process
import numpy as np
import re
import sqlalchemy.exc as sa_exc
from sqlalchemy.exc import SQLAlchemyError
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

## VISUALIZACIÓN

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# EXTRACT

In [54]:
df = pd.read_excel(r"C:\Users\aleja\OneDrive\Documents\Oracle - Program\orange\01 OVERALL FINAL DATA PHD.xlsx",
                   sheet_name='RAW DATA', )

# TRANSFORM

## SEPARAR LAS EMPRESAS DE LAS MEDICIONES

In [ ]:
companies = df.iloc[7:, :28] 
companies.columns = companies.iloc[0]
companies = companies[2:].reset_index(drop=True)
companies

In [145]:
scores = df.iloc[:, 27:]
scores = scores.iloc[1:]

## ORGANIZAR EL DF

In [146]:
# Paso 1: Identificar las columnas donde alguna fila contiene 'Addressed criteria'
columnas_a_mover = scores.columns[scores.apply(lambda col: col.astype(str).str.contains('Addressed criteria', na=False)).any()]

# Paso 2: Crear un nuevo DataFrame con estas columnas
addressed_criteria = scores[columnas_a_mover].copy()

# Paso 3: Eliminar estas columnas del DataFrame original
scores.drop(columns=columnas_a_mover, inplace=True)

In [147]:
# Paso 1: Identificar las columnas donde alguna fila contiene 'Use of tools'
columnas_a_mover = scores.columns[scores.apply(lambda col: col.astype(str).str.contains('Use of tools', na=False)).any()]

# Paso 2: Crear un nuevo DataFrame con estas columnas
use_tools = scores[columnas_a_mover].copy()

# Paso 3: Eliminar estas columnas del DataFrame original
scores.drop(columns=columnas_a_mover, inplace=True)

### TRANSPONER EL DF

In [148]:
# Eliminar las filas 1, 4 y 7
scores = scores.reset_index(drop=True).drop(index=[1, 4, 7]).reset_index(drop=True)

### CONCATENARLE EL CÓD DE LA EMPRESA

In [105]:
concat_companies = companies[['CODE']]

In [106]:
# Crear 5 filas nulas con las mismas columnas que el DataFrame original
filas_nulas = pd.DataFrame([[None] * concat_companies.shape[1]] * 5, columns=concat_companies.columns)

# Concatenar las filas nulas con el DataFrame original
concat_companies = pd.concat([filas_nulas, concat_companies], ignore_index=True)

In [149]:
scores = pd.concat([concat_companies,scores],axis=1)

In [ ]:
# scores.drop(columns=['CODE'],inplace=True)

In [142]:
# Paso 1: Transponer el DataFrame (convertir filas en columnas)
dfT = scores.T.reset_index(drop=True)

# Paso 2: Renombrar las columnas para que representen los niveles de jerarquía
dfT.columns = ["Dynamic_Capability_Stage", "Business_Parameters", "Business_Sub", "Metrics","Question"] + [f"Valor_{i}" for i in range(len(dfT.columns)-5)]

# Paso 3: Convertir a formato largo (melt)
df_long = dfT.melt(id_vars=["Dynamic_Capability_Stage", "Business_Parameters", "Business_Sub", "Metrics","Question"], 
                   var_name="Medicion", value_name="Valor")

# Paso 4: Eliminar la columna de "Medicion" ya que no aporta información
df_long = df_long.drop(columns=["Medicion"])

In [151]:
# Paso 2: Asegurar que la primera fila sea los nombres de columna
dfT.columns = dfT.iloc[0]  # Asignar la primera fila como nombres de columnas
dfT = dfT.iloc[1:].reset_index(drop=True)  # Eliminar la fila de encabezado y resetear índice

In [153]:
dfT.to_excel("transpuesta.xlsx", index=False)